In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd 
from pulp import * 
import seaborn as sns

In [2]:
# Navigate to dataset directory
current_working_directory = os.getcwd()

# Set location to Datasets
dataset_location = current_working_directory + '\Datasets\Food_recommender\\'

print(dataset_location)

d:\Varshini\CourseWork\Dissertation\Implementation\Github\Recommendation_Dissertation\Datasets\Food_recommender\


In [3]:
# Read the nutrition csv
nutrition_data = pd.read_csv(dataset_location + 'nutrition.csv')
nutrition_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'd:\\Varshini\\CourseWork\\Dissertation\\Implementation\\Github\\Recommendation_Dissertation\\Datasets\\Food_recommender\\nutrition.csv'

In [20]:
# Data cleaning, select columns
# Select columns: name, calories, cholesterol, folic_acid, vitamin_c, vitamin_d, calcium, iron, protein, carbohydrate, fiber, sugars, fat
subset_nutrition_data = nutrition_data[["name", "calories", "cholesterol", "folic_acid", "vitamin_c", "vitamin_d", "calcium", "iron", "protein", "carbohydrate", "fiber", "sugars", "fat"]]

subset_nutrition_data.isnull().sum()
subset_nutrition_data.isna().sum() # No nan or null values

# Replace strings in columns
subset_nutrition_data = subset_nutrition_data.replace('mcg', '', regex=True)
subset_nutrition_data = subset_nutrition_data.replace('mg', '', regex=True)
subset_nutrition_data = subset_nutrition_data.replace('g', '', regex=True)
subset_nutrition_data = subset_nutrition_data.replace('IU', '', regex=True)

# Covert columns to float dtype
subset_nutrition_data = subset_nutrition_data.astype({'cholesterol':'float',
                                                      'folic_acid':'float',
                                                      'vitamin_c':'float',
                                                      'vitamin_d':'float',
                                                      'calcium':'float',
                                                      'iron':'float',
                                                      'protein':'float',
                                                      'carbohydrate':'float',
                                                      'fiber':'float',
                                                      'sugars':'float',
                                                      'fat':'float'})

# Modify column names to include the unit
subset_nutrition_data.rename(columns = {'cholesterol':'cholesterol(mg)', 
                                        'folic_acid':'folic_acid(mcg)',
                                        'vitamin_c':'vitamin_c(mg)',
                                        'vitamin_d':'vitamin_d(IU)',
                                        'calcium':'calcium(mg)',
                                        'iron':'iron(mg)',
                                        'protein':'protein(g)',
                                        'carbohydrate':'carbohydrate(g)',
                                        'fiber':'fiber(g)',
                                        'sugars':'sugars(g)',
                                        'fat':'fat(g)'}, inplace = True)



subset_nutrition_data.head()


,name,calories,cholesterol(mg),folic_acid(mcg),vitamin_c(mg),vitamin_d(IU),calcium(mg),iron(mg),protein(g),carbohydrate(g),fiber(g),sugars(g),fat(g)
0,Cornstarch,381,0.0,0.0,0.0,0.0,2.0,0.47,0.26,91.27,0.9,0.00,0.05
1,"Nuts, pecans",691,0.0,0.0,1.1,0.0,70.0,2.53,9.17,13.86,9.6,3.97,71.97
2,"Eplant, raw",25,0.0,0.0,2.2,0.0,9.0,0.23,0.98,5.88,3.0,3.53,0.18
3,"Teff, uncooked",367,0.0,0.0,0.0,0.0,180.0,7.63,13.30,73.13,8.0,1.84,2.38
4,"Sherbet, orane",144,1.0,0.0,2.3,0.0,54.0,0.14,1.10,30.40,1.3,24.32,2.00


In [22]:
week_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
split_values = np.linspace(0,len(subset_nutrition_data),8).astype(int)
split_values[-1] = split_values[-1]-1
def random_dataset():
    frac_data = subset_nutrition_data.sample(frac=1).reset_index().drop('index',axis=1)
    day_data = []
    for s in range(len(split_values)-1):
        day_data.append(frac_data.loc[split_values[s]:split_values[s+1]])
    return dict(zip(week_days,day_data))

random_dataset()

,name,calories,cholesterol(mg),folic_acid(mcg),vitamin_c(mg),vitamin_d(IU),calcium(mg),iron(mg),protein(g),carbohydrate(g),fiber(g),sugars(g),fat(g)
0,"Lamb, raw, separable lean and fat, shank off, ...",121,64.0,0.0,0.0,0.0,4.0,1.40,20.93,0.00,0.0,0.00,4.09
1,"Chicken, raw, meat only, liht meat, broilers o...",114,58.0,0.0,0.0,0.0,12.0,0.73,23.20,0.00,0.0,0.00,1.65
2,"Beef, simmered, cooked, tripe, variety meats a...",94,157.0,0.0,0.0,0.0,81.0,0.66,11.71,1.99,0.0,0.00,4.05
3,"Candies, milk chocolate coated coffee beans",549,20.0,0.0,0.0,0.0,169.0,2.29,7.41,55.25,5.7,50.75,33.18
4,"Refried beans, veetarian, canned",83,0.0,0.0,0.0,0.0,35.0,1.70,5.28,13.50,4.7,0.59,0.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251,"Chicken, roasted, cooked, meat only, back, bro...",239,90.0,0.0,0.0,5.0,24.0,1.39,28.19,0.00,0.0,0.00,13.16
1252,"Beef, braised, cooked, separable lean and fat,...",453,72.0,0.0,0.0,10.0,11.0,1.29,20.14,0.00,0.0,0.00,41.33
1253,"Beef, rilled, cooked, all rades, trimmed to 0""...",182,77.0,0.0,0.0,0.0,5.0,2.99,27.52,0.00,0.0,0.00,7.10
1254,"Rice, enriched, raw, reular, lon-rain, white",365,0.0,223.0,0.0,0.0,28.0,4.31,7.13,79.95,1.3,0.12,0.66
